# Evaluating models on the dataset

- Caricare i video
- Preprocessing dei video (resize e normalizzazione)
- Split train test
- Estrazione features con CNN
- passaggio features a LSTM per classificazione

## Imports

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from wp8.pre_processing.utils import listdir_nohidden_sorted as lsdir
from tqdm.notebook import tqdm
import re
from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from my_generators import TimeSeriesGenerator as TSG
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn import preprocessing
from tensorflow.keras.callbacks import TensorBoard
import wandb
from wandb.keras import WandbCallback

## Load dataset and features

In [ ]:
features_path = "../outputs/dataset/features/"
dataset_path = "../outputs/dataset/dataset/"

#load features
all_features = []
all_features_paths = lsdir(features_path)[0:1]
for _, feature_file in enumerate(tqdm(all_features_paths)):
  with np.load(feature_file) as features:
      all_features.append(features["arr_0"])
      
all_features=np.concatenate(all_features, axis=0)

In [ ]:
dfs = []
for _,filename in enumerate(tqdm(lsdir(dataset_path)[0:1])):
  df = pd.read_csv(filename, index_col=0)
  dfs.append(df)

dataset = pd.concat(dfs, ignore_index=True)

In [ ]:
# #save full dataset and features
# ssd_path = "/Volumes/SSD 1TB 1/Alt Frailty WP8/DATASET_processed"
# dataset.to_csv(f"{ssd_path}/dataset.csv")

# np.save(f"{ssd_path}/features", all_features)

In [ ]:
print(dataset.shape, all_features.shape)

In [ ]:
dataset.head(-10)

In [ ]:
names = dataset["frame_name"]
cams = []
for name in names:
  cams.append(int(name[-6]))

dataset["cams"] = pd.Series(cams)

dataset

In [ ]:
dataset["features"] = pd.Series(all_features.tolist())

In [ ]:
print(dataset["features"].isna().sum())

In [ ]:
#count samples per label
dataset["micro_labels"].value_counts()

In [ ]:

shape = (dataset["features"].shape[0], 2048)
features = np.array(dataset["features"].to_list())
type(features[0])

In [ ]:
# le = preprocessing.LabelEncoder()
# batch = (features, le.fit_transform(dataset["micro_labels"]), dataset["cams"].tolist())
# gen = TSG(X = batch[0], y = batch[1], cams=batch[2], num_features=2048, batch_size=32, stride=10, seq_len = 20)

In [ ]:
le = preprocessing.LabelEncoder()
encoded_labels = le.fit_transform(dataset["micro_labels"])
n_labels = len(np.unique(encoded_labels))
print(n_labels)

In [ ]:
%load_ext tensorboard
import datetime
log_folder = "../logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")


In [ ]:
tensorboard_callback=TensorBoard(log_dir=log_folder)

gen = TSG(X=features, y = encoded_labels, num_features=2048, cams=dataset["cams"].tolist(), batch_size = 32, stride=20, seq_len = 20)

model = Sequential()
model.add(LSTM(units=128, input_shape=(20, 2048)))
model.add(Dense(n_labels, activation = "softmax"))
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics = ["accuracy"])
history=model.fit(gen, epochs=1, use_multiprocessing=True)


In [ ]:
%tensorboard --logdir log_folder